In [5]:
import os
import sct_utils as sct
import numpy as np
from msct_image import Image
from sct_deepseg_lesion import crop_image_around_centerline 
from sct_deepseg_lesion import apply_intensity_normalization

In [6]:
# Specifying input data
PATH_nii='/Users/pruthvi_local/Desktop/Lesion_Segmentation/Only_NIIs/'
PATH_mask = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/Masks/'
subject_masks=os.listdir(PATH_mask)
Mask_list=[x.split('.')[0] for x in subject_masks]
# print(subject_nii)
# print(subject_list)

for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]

In [7]:
# # 2.1 --Second option of getting centerline -- 2.1

for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_nii + subject + '.nii'
    out_path = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/01deepseg_sc'
    sct.run(['sct_deepseg_sc', '-i', img_path, '-c', 't2', '-ofolder', out_path], verbose=0)


PATH_deep_seg = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/01deepseg_sc/'
out_deep_seg_path = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/02sct_process_seg/' 

# Second option of getting centerline -- 2.2
# Specifying input data
for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_deep_seg + subject + '_seg.nii'
    print(img_path) 
    try: 
        sct.run(['sct_process_segmentation', '-i', img_path, '-p', 'centerline', '-ofolder', out_deep_seg_path], verbose=0)
    except:
        continue    

In [8]:
# 3.0 Reorientation of original image and ROI

for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_nii + subject + '.nii'    
    print img_path   
    try: 
        sct.run(['sct_image', '-i', img_path, ' -setorient RPI'], verbose=0)
    except:
        continue   

# Specifying roi data
PATH_process_seg = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/02sct_process_seg/'
  
for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_process_seg + subject + '_seg_centerline.nii.gz'
    print img_path   
    try: 
        sct.run(['sct_image', '-i', img_path, ' -setorient RPI'], verbose=0)
    except:
        continue 


# Here specify LESION MASK DATA
for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_mask + subject + '_mask.nii'
    print img_path   
    try: 
        sct.run(['sct_image', '-i', img_path, ' -setorient RPI'], verbose=0)
    except:
        continue 


In [26]:
# 4.0 Resampling of image

# Resample Input image
out_path = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/'
 
for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_nii + subject + '.nii'
    img_out_path = out_path + subject + '_resample_INPUT.nii.gz'   
    print(img_out_path)
    try: 
        sct.run(['sct_resample', '-i', img_path, '-x', 'nn', '-mm', '0.5x0.5x0.5', '-o', img_out_path], verbose=0)
    except:
        continue 
      
     
# Resample centerline mask   

for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_process_seg + subject + '_seg_centerline.nii.gz'
    img_out_path = out_path + subject + '_resample_centerline.nii.gz' 
    print(img_out_path)
    try: 
#         sct.run(['sct_resample', '-i', img_path, '-mm', '0.5x0.5x0.5', '-o', img_out_path], verbose=0)
        sct.run(['sct_resample', '-i', img_path, '-x', 'nn', '-mm', '0.5x0.5x0.5', '-o', img_out_path], verbose=0)

    except:
        continue 


        
# Resample LESION MASK DATA

for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_path = PATH_mask + subject + '_mask.nii'
    img_out_path = out_path + subject + '_resample_mask.nii.gz'   
    print(img_out_path)
    try: 
#         sct.run(['sct_resample', '-i', img_path, '-mm', '0.5x0.5x0.5', '-o', img_out_path], verbose=0)
        sct.run(['sct_resample', '-i', img_path, '-x', 'nn', '-mm', '0.5x0.5x0.5', '-o', img_out_path], verbose=0)
    except:
        continue 

FileNotFoundError: No such file: '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/aw5_resample_centerline.nii.gz'

In [27]:
# 5.0 Cropping image
out_path = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/04crop/'
PATH_resample = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/03Resample/'
PATH_process_seg = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/02sct_process_seg/'
PATH_nii='/Users/pruthvi_local/Desktop/Lesion_Segmentation/Only_NIIs/'
PATH_mask = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/Masks/'


for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    
    NII_path = PATH_resample + subject + '_resample_INPUT.nii.gz'
    MASK_path = PATH_resample + subject + '_resample_MASK.nii.gz'
    ctr_path = PATH_resample + subject + '_resample_centerline.nii.gz'  

    img_NII_path = out_path + subject + '_crop_INPUT.nii'
    img_MASK_path = out_path + subject + '_crop_MASK.nii'     
    
    try: 
        crop_image_around_centerline(NII_path, ctr_path, img_NII_path, 48)
        crop_image_around_centerline(MASK_path, ctr_path, img_MASK_path, 48)
        print(NII_path)
        print(MASK_path)
        print(ctr_path)
    except:
        continue     

In [28]:
# Standardize image intensities
in_path = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/04crop/'
ofolder_path = '/Users/pruthvi_local/Desktop/Lesion_Segmentation/01_preprocess/05Norm_Int/'
for line in Mask_list:
    Type = line.split("_")
    subject = Type[0]
    img_in_path = in_path + subject + '_crop_INPUT.nii'
    img_out_path = ofolder_path + subject + '_norm.nii'
    try: 
        apply_intensity_normalization(img_in_path, img_out_path, 't2')    
    except:
        continue        